### Libraries

In [2]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.spatial.distance import mahalanobis
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score # TODO 

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV #TODO
from sklearn.linear_model import LinearRegression # TODO
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR #TODO

### Set random seed

In [3]:
# set the random state
random_state = min(332078,332464)
np.random.seed(random_state)

### Dataset

In [4]:
df_dev = pd.read_csv('development.csv')

In [ ]:
# shape
df_dev.shape

In [ ]:
# columns
df_dev.columns

1) x, y: the position of the events over the sensor
2) pmax[0], pmax[1], ... pmax[17]: the magnitude of the positive peak of the signal, in mV
3) negpmax[0], negpmax[1], ... negpmax[17]: the magnitude of the negative peak of the signal, in mV
4) tmax[0], tmax[1], ... tmax[17]: the delay (in ns) from a reference time when the positive peak of the signal
5) area[0], area[1], ... area[17]: the area under the signal
6) rms[0], rms[1], ... rms[17]: the root mean square (RMS) value of the signal

In [ ]:
# NaN values
display(df_dev.isna().any())
print(f'\nThe dataset has {df_dev.isna().any().sum()} NaN values')

# Preprocessing

### Data visualization

In [ ]:
# scatter plot of x and y
df_dev.plot.scatter('x','y', s=20, alpha=0.5)
plt.xlabel('$X$')
plt.ylabel('$Y$',rotation=0)
plt.title('Events')
plt.show()

Some areas of the sensor are not covered by any event. That occurs because, at those coordinates, either pads or wires used to read the signals from the pads (due to their reflective properties) are presen

In [ ]:
# plot of the features
for elem in ['pmax','negpmax','area','tmax','rms']:
    rows, cols = 6, 3
    fig, ax = plt.subplots(rows, cols, figsize=(25, 20))
    plt.subplots_adjust(wspace=0.5, hspace=0.5)
    i=0
    for row in range(rows):
        for col in range(cols):
            ax[row, col].plot(df_dev[f'{elem}[{i}]'])
            ax[row,col].set_title(f'{elem}[{i}]')
            ax[row,col].set_xlabel('events')
            ax[row,col].set_ylabel('magnitude')
            i+=1
    plt.show()
    print()

In [ ]:
# sensor position with respect to the pmax feature
elem = 'pmax'
rows, cols = 6, 3
fig, ax = plt.subplots(rows, cols, figsize=(25, 20))
plt.subplots_adjust(wspace=0.5, hspace=0.5)
i=0
for row in range(rows):
    for col in range(cols):
        limit = df_dev[f'{elem}[{i}]'].mean() + 2*df_dev[f'{elem}[{i}]'].std()
        mask = df_dev[f'{elem}[{i}]'] > limit
        ax[row, col].scatter(df_dev['x'],df_dev['y'], s=15, alpha=0.5)
        ax[row, col].scatter(df_dev[mask]['x'],df_dev[mask]['y'],c='orange', s=15)
        ax[row,col].set_title(f'Sensor [{i}]')
        ax[row,col].set_xlabel('x')
        ax[row,col].set_ylabel('y',rotation=0)
        ax[row, col].set_xlim(200, 600)
        ax[row, col].set_ylim(200, 600)
        i+=1
plt.show()

### Remove noise features

Thanks to the previous graph, we can deduce that the noise is caused by features: 0, 7, 12, 15, 16, and 17. As for feature 15, it was not easy to determine whether it was actually a source of noise. However, thanks to the last graph, we can see that the alleged pad 15 is able to detect high peaks on distant sensor areas. This implies that it does indeed constitute a source of noise.

In [5]:
# list of feature indices with noise
noise_features = [0,7,12,15,16,17]
# list of feature indices without noise
no_noise_features = [elem for elem in np.arange(18) if elem not in noise_features]

In [6]:
# remove noise features
for i in noise_features:
   df_dev = df_dev.drop(columns=[f'pmax[{i}]',f'negpmax[{i}]',f'area[{i}]',f'tmax[{i}]',f'rms[{i}]'])

### Outlier detection

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 15))

# first row
for idx, elem in enumerate(['pmax', 'negpmax']):
    df_dev[[f'{elem}[{i}]' for i in no_noise_features]].boxplot(ax=axes[0, idx])
    axes[0, idx].set_xticklabels(axes[0, idx].get_xticklabels(), rotation=75)
    axes[0, idx].set_title(elem)

# second row
for idx, elem in enumerate(['area', 'tmax']):
    df_dev[[f'{elem}[{i}]' for i in no_noise_features]].boxplot(ax=axes[1, idx])
    axes[1, idx].set_xticklabels(axes[1, idx].get_xticklabels(), rotation=75)
    axes[1, idx].set_title(elem)

# third row
for idx, elem in enumerate(['rms']):
    df_dev[[f'{elem}[{i}]' for i in no_noise_features]].boxplot(ax=axes[2, idx])
    axes[2, idx].set_xticklabels(axes[2, idx].get_xticklabels(), rotation=75)
    axes[2, idx].set_title(elem)

# remove empty subplot in the third row
fig.delaxes(axes[2, -1])

plt.tight_layout()
plt.show()

In [7]:
# remove positive negpmax values
print('Number of lines before removing positive values from negpmax:', df_dev.shape[0])

for i in no_noise_features:
    df_dev = df_dev[df_dev[f'negpmax[{i}]'] < 0]
print('Number of lines after removing positive values from negpmax:', df_dev.shape[0])

Number of lines before removing positive values from negpmax: 385500
Number of lines after removing positive values from negpmax: 385497


##### ZSCORE OVER SEPARATE SENSORS

In [8]:
df_sensor = df_dev.copy()

for i in no_noise_features:
    list_values = []
    for elem in ['pmax','negpmax','area','tmax','rms']:
        values = df_sensor[[f'{elem}[{i}]']].values
        list_values.append((values - values.mean()) / values.std())
    df_sensor[f'zscore[{i}]'] = np.array(list_values).mean(axis=0)

In [9]:
zscores = [col for col in df_sensor.columns if col.startswith('zscor')]

In [10]:
from collections import Counter
def get_index_zscore(df, threshold):
    return df[(df > threshold) | (df < -threshold)].index

threshold = 0.2
index_dict = {f'index_s{i}zscore': get_index_zscore(df_sensor[f'zscore[{i}]'], threshold) for i in no_noise_features}

index_list_zscore = np.concatenate([index for _, index in index_dict.items()])
sensor_counter = Counter(index_list_zscore)
index_to_remove = [key for key, count in sensor_counter.items() if count == 12]

last_dim = df_no_outliers.shape[0]
df_no_outliers = df_sensor.drop(index_to_remove).drop(columns=zscores)

print(f'Non-outlier observations: {df_no_outliers.shape[0]}')
print(f'Identified outliers: {last_dim - df_no_outliers.shape[0]}')
print()

In [ ]:
df_no_outliers

##### MAHALANOBIS DISTANCE 

In [ ]:
df_no_outliers = pd.DataFrame(df_dev)

# calculate mean and inverse covariance matrix
mean = df_no_outliers.drop(columns=['x','y']).mean()

covariance_matrix = df_no_outliers.drop(columns=['x','y']).cov()

cov_inv = pd.DataFrame(np.linalg.inv(covariance_matrix.values), 
                       columns = df_no_outliers.drop(columns=['x','y']).columns, 
                       index = df_no_outliers.drop(columns=['x','y']).columns)
# calculate Mahalanobis distance for each data point
df_no_outliers['mahalanobis_distance'] = df_no_outliers.drop(columns=['x','y']).apply(lambda x: mahalanobis(x, mean, cov_inv), axis=1)

In [ ]:
threshold = df_no_outliers['mahalanobis_distance'].mean() +  3.5 * df_no_outliers['mahalanobis_distance'].std()

In [ ]:
# plot the mahalanobis_distance over the events
df_no_outliers['mahalanobis_distance'].plot(linewidth=0.25, label='Mahalanobis Distance')
plt.axhline(y=threshold, color='red', linestyle='--', label='Threshold')
plt.show()

In [ ]:
# plot the Mahalanobis Distance distribution
plt.hist(df_no_outliers['mahalanobis_distance'], bins=150, color='blue', alpha=0.7)
plt.axvline(x=threshold, color='red', linestyle='--', label='Threshold')

# Set title and labels
plt.title('Mahalanobis Distance Distribution with Threshold')
plt.xlabel('Mahalanobis Distance')
plt.ylabel('Frequency')

plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_no_outliers = df_no_outliers[df_no_outliers['mahalanobis_distance'] < threshold]

In [ ]:
385497 - df_no_outliers.shape[0] 

In [ ]:
df_no_outliers = df_no_outliers.drop(columns='mahalanobis_distance')

##### ZSCORE OUTLIER

In [ ]:
df_no_outliers = pd.DataFrame(df_dev)

n_outliers = 0
for elem in ['pmax','negpmax','area','tmax','rms']:
    for i in no_noise_features:
        values = df_no_outliers[[f'{elem}[{i}]']]
        df_no_outliers['zscore'] = (values - values.mean()) / values.std()

        l=15
        
        outliers = df_no_outliers[(df_no_outliers['zscore']< -l) | (df_no_outliers['zscore']> l)]
        n_outliers += len(outliers)
        
        df_no_outliers = df_no_outliers[(df_no_outliers['zscore'] >= -l) & (df_no_outliers['zscore']<= l)]
        
df_no_outliers = df_no_outliers.drop(columns=['zscore'])
print('Non-outlier observations: %d' % len(df_no_outliers))
print('Identified outliers: %d' % n_outliers)
print()

In [ ]:
df_no_outliers.shape

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(12, 15))

# Boxplots for the first row
for idx, elem in enumerate(['pmax', 'negpmax']):
    df_dev[[f'{elem}[{i}]' for i in no_noise_features]].boxplot(ax=axes[0, idx])
    axes[0, idx].set_xticklabels(axes[0, idx].get_xticklabels(), rotation=75)
    axes[0, idx].set_title(elem)

# Boxplots for the second row
for idx, elem in enumerate(['area', 'tmax']):
    df_dev[[f'{elem}[{i}]' for i in no_noise_features]].boxplot(ax=axes[1, idx])
    axes[1, idx].set_xticklabels(axes[1, idx].get_xticklabels(), rotation=75)
    axes[1, idx].set_title(elem)

# Boxplots for the third row
for idx, elem in enumerate(['rms']):
    df_dev[[f'{elem}[{i}]' for i in no_noise_features]].boxplot(ax=axes[2, idx])
    axes[2, idx].set_xticklabels(axes[2, idx].get_xticklabels(), rotation=75)
    axes[2, idx].set_title(elem)

# Remove empty subplot in the third row
fig.delaxes(axes[2, -1])

plt.tight_layout()
plt.show()

### Features selection

In [ ]:
# train test split
X = df_no_outliers.drop(columns=['x', 'y']).values
y = df_no_outliers[['x', 'y']].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, shuffle=True, random_state=random_state)

In [ ]:
# Random Forest x feature selection
reg = RandomForestRegressor(n_estimators=100, random_state=random_state, n_jobs=-1)

In [ ]:
# start timer
start_time = time.time()

# fit the model
reg.fit(X_train , y_train)

# end timer
end_time = time.time()
print(f'Execution Time: {end_time - start_time} seconds')

In [ ]:
# R2 score
print('R^2 score:',r2_score(y_valid, reg.predict(X_valid)))

In [ ]:
# feature importances
feature_names = df_no_outliers.drop(columns=['x', 'y']).columns

feature_importances_list = sorted(zip(feature_names, reg.feature_importances_), key = lambda x: x[1], reverse = True)
feature_importances_list

In [ ]:
# plot the feature importances

# extract feature names and importances from the list
feature_names, importances = zip(*feature_importances_list)

# Plotting the vertical bar chart
plt.figure(figsize=(15, 5))
plt.bar(feature_names, importances)
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.xticks(rotation=75, ha='right')  # Rotate x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Show the plot
plt.show()

As indicated by the importance given to the different features, it is evident that the worst categories are represented by tmax, rms, and area. Therefore, we will proceed with the removal of these features in order to reduce the size of the dataset.

In [11]:
# removing tmax, area and rms feature
for i in no_noise_features:
    df_no_outliers = df_no_outliers.drop(columns=[f'tmax[{i}]',f'rms[{i}]', f'area[{i}]'])

In [12]:
# rename the dataset
df_clean = df_no_outliers

# Model implementation

In [13]:
# Train test splitting
X = df_clean.drop(columns=['x', 'y']).values
y = df_clean[['x', 'y']].values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, shuffle=True, random_state=random_state)

In [ ]:
# Train test scale and splitting 

# standard scaler
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_train_scaled, X_valid_scaled, y_train_scaled, y_valid_scaled = train_test_split(X_scaled, y, test_size= 0.2, shuffle=True, random_state=random_state)

### Random Forest

In [ ]:
# set the params
param_grid = {
    'n_estimators': [i for i in range(800,1100,100)],
    'criterion': ['squared_error'],
    'max_features': ['sqrt'],
    'random_state': [random_state],
}

gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring='r2', n_jobs=-1, cv=5)

#start
start_time = time.time()

gs.fit(X_train, y_train)

#end
end_time = time.time()

print(f'Execution Time: {end_time - start_time} seconds')
print(f'GridSearchCV best params: {gs.best_params_}')

######## [i for i in range(100,500,100)] ################################
# Execution Time: 1421.2425608634949 seconds 
# GridSearchCV best params: {'criterion': 'squared_error', 
#                            'max_features': 'sqrt', 
#                            'n_estimators': 400, 
#                            'random_state': 332078}
#
#                      ------- score: 4.918 -------
#
##########################################################################

######## [i for i in range(500,800,100)] ################################
#Execution Time: 2413.7543437480927 seconds
# GridSearchCV best params: {'criterion': 'squared_error', 
#                            'max_features': 'sqrt', 
#                            'n_estimators': 700, 
#                            'random_state': 332078}
#
#                      ------- score: 4.903 -------
#
##########################################################################

######## [i for i in range(800,1100,100)] ################################
# GridSearchCV best params: {'criterion': 'squared_error', 
#                            'max_features': 'sqrt', 
#                            'n_estimators': 1000, 
#                            'random_state': 332078}
#
#                      ------- score: 4.899 -------
#
##########################################################################

In [ ]:
reg = RandomForestRegressor(n_estimators=2500, 
                            criterion='squared_error', 
                            max_features='sqrt', 
                            random_state=random_state)

#start training
start_time = time.time()

reg.fit(X_train , y_train)

#end training
end_time = time.time()
print(f'Execution Time: {end_time - start_time} seconds')

# predict
y_pred = reg.predict(X_valid)

In [ ]:
# predict
reg = gs.best_estimator_
# predict
y_pred = reg.predict(X_valid)

### KNN

In [ ]:
# chose the best weights and algorithm
param_grid = {'n_neighbors': [i for i in range(5,50,5)],
              'weights': ['uniform', 'distance'],
              'algorithm':['auto', 'ball_tree', 'kd_tree'],
             }


gs = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')

#start training 
start_time = time.time()

gs.fit(X_train, y_train)

#end training
end_time = time.time()

print(f'Execution Time: {end_time - start_time} seconds')
print(f'GridSearchCV best params: {gs.best_params_}')

In [ ]:
# pick the best model
kkn = gs.best_estimator_

In [ ]:
# make the local prediction
y_pred = kkn.predict(X_valid)

After applying grid search on the model we saw that the best model parameters are: {}.

With this model we obtained a score, considering the Euclidean distance, of 5.573, which is worse than that obtained with the RandomForestRegressor

### SVR

In [ ]:
# SVR model
svr = SVR()

# Multi output wrapper
multi_svr = MultiOutputRegressor(svr)

#start
start_time = time.time()

multi_svr.fit(X_train_scaled , y_train_scaled)

#end
end_time = time.time()
print(f'Execution Time: {end_time - start_time} seconds')

y_pred = wrapper.predict(X_valid_scaled)

In [ ]:
# Importa le librerie necessarie
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import numpy as np

# Assume che 'X' sia il tuo array di features (pmax, negpmax, area, rms, tmax per ogni sensore)
# e 'y' sia il tuo array bidimensionale di coordinate (x, y)

# Divide il dataset in training e test set
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, shuffle=True, random_state=random_state)

# Crea una pipeline con uno scaler e un regressore SVR
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalizza le feature
    ('svr', MultiOutputRegressor(SVR()))  # Utilizza il Support Vector Regressor
])

# Addestra il modello sulla pipeline con i dati di training
pipeline.fit(X_train, y_train)


In [ ]:
y_pred = pipeline.predict(X_valid)

# Calcola l'errore quadratico medio (MSE) per valutare le prestazioni
mse = mean_squared_error(y_valid, y_pred)
print(f'Mean Squared Error: {mse}')

In [ ]:
pipeline

# Local evaluation

In [ ]:
# evaluate the model through Euclidean distance
def distance_evaluation(y_true, y_pred):
    # distance
    distances = np.sqrt(np.sum((y_true - y_pred)**2, axis=1))
    # distance mean
    result = np.mean(distances)
    return result

result = distance_evaluation(y_valid, y_pred)
print('Distance Evaluation Result:', result)

In [ ]:
#Distance Evaluation Result: 4.297281186491101
 #con 1000 estimator

In [ ]:
...

# Export the results

In [ ]:
# import evaluation dataaset
df_eval = pd.read_csv('evaluation.csv',index_col='Id')

In [ ]:
# apply the same transformations to the evaluation dataset as applied to the development dataset

# remove noise features
for i in noise_features:
   df_eval = df_eval.drop(columns=[f'pmax[{i}]',f'negpmax[{i}]',f'area[{i}]',f'tmax[{i}]',f'rms[{i}]'])

# remove the less important features
for i in no_noise_features:
    df_eval = df_eval.drop(columns=[f'tmax[{i}]',f'rms[{i}]',f'area[{i}]'])

In [ ]:
# make the predictions
X_eval = df_eval.values

y_pred = reg.predict(X_eval)

In [ ]:
# apply the correct format for evaluation
df_pred = pd.DataFrame(y_pred, columns=['Predicted1','Predicted2'])

df_pred['Id'] = df_pred.index

df_pred['Predicted'] = df_pred[['Predicted1', 'Predicted2']].astype(str).agg('|'.join, axis=1)

df_pred = df_pred.drop(columns=['Predicted1', 'Predicted2'])

df_pred.to_csv('pred.csv', index=False)